# Parallelizing Image Convolution

## Learning Goals

By the end of this tutorial, you will be able to:

- Employ three parallelization libraries to speed up a serial process.
- Calculate the speedup of the different approaches shown.
- Evaluate which library is suited to your task.

## Introduction

This notebook shows how to speed up an image convolution task using these three libraries:

* Ray: an open-source unified compute framework that makes it easy to scale AI and Python workloads.
* Multiprocessing: part of the standard library; supports spawning processes using an API similar to the threading module; offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads.
* Dask: developed to natively scale computational packages like numpy, pandas and scikit-learn, and the surrounding ecosystem, to multi-core machines and distributed clusters when datasets exceed memory.

## Imports

* _multiprocessing.Pool_ for multiprocessing using the standard library
* _time_ for timing the processes
* _dask.distributed.Client_ for making a local Dask cluster
* _numpy_ and _scipy.signal_ for numerical work
* _psutil_ for finding the available processors on your machine
* _ray_ for scaling up Python tasks

In [1]:
from multiprocessing import Pool
import time

from dask.distributed import Client
import numpy as np
import psutil
import scipy.signal
import ray

2024-08-29 17:08:24,355	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Find the cpus available

Find and print the number of cpus
(taken from https://towardsdatascience.com/10x-faster-parallel-python-without-python-multiprocessing-e5017c93cce1)

In [2]:
num_cpus = psutil.cpu_count(logical=True)
print(num_cpus)

4


## Process serially using a conventional loop

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result.

In [3]:
def fconv(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In [4]:
filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]

Process 100 iterations serially, then extrapolate to num_cpus*100

In [5]:
start = time.time()
num_iter = 100
image = np.zeros((3000, 3000))
for i in range(num_iter):
    result = fconv(image, filters[i % num_cpus])
duration_conv = time.time() - start
print("(scaled) conventional duration for {:d} iterations = {:.1f} seconds"
      .format(num_cpus*num_iter, duration_conv*num_cpus))

(scaled) conventional duration for 400 iterations = 141.2 seconds


## Process in parallel using Ray

[Documentation for ray](https://docs.ray.io/en/latest/)

The warning raised by `ray.init` only affects shared object usage, which is not an issue for this tutorial. It may harm performance in other scenarios.

In [6]:
ray.init(num_cpus=num_cpus)

2024-08-29 17:09:01,805	INFO worker.py:1783 -- Started a local Ray instance.


Python version:,3.11.9
Ray version:,2.35.0


Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. To use Ray, we decorate the function that is doing the work.

In [7]:
@ray.remote
def fray(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In the following loop, `ray.put` places the image into shared memory. The call to `ray.get` retrieves the result.

In [8]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    image_id = ray.put(image)
    ray.get([fray.remote(image_id, filters[i]) for i in range(num_cpus)])
duration_ray = time.time() - start
print("Ray duration = {:.1f}, speedup = {:.2f}"
      .format(duration_ray, duration_conv*num_cpus / duration_ray))

Ray duration = 82.4, speedup = 1.71


In [9]:
ray.shutdown()

## Process in parallel using multiprocessing

[Documentation for multiprocessing](https://docs.python.org/3/library/multiprocessing.html)

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. The call to the function has a slightly different form than that for the serial loop.

In [10]:
def fmp(args):
    image, random_filter = args
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

Use a multiprocessing pool with the number of cpus we found earlier.

In [11]:
pool = Pool(num_cpus)

Using `pool.map` is the closest analog in multiprocessing to the Ray API.

In [12]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    pool.map(fmp, zip(num_cpus * [image], filters))
duration_mp = time.time() - start
print("Multiprocessing duration = {:.1f}, speedup = {:.2f}"
      .format(duration_mp, duration_conv*num_cpus / duration_mp))

Multiprocessing duration = 139.1, speedup = 1.02


## Process using Dask

[Documentation for Dask](https://www.dask.org/get-started)

Define a Dask distributed client with number of workers set to the number of cpus we found earlier, and with one thread per worker.

In [13]:
client = Client(n_workers=num_cpus, threads_per_worker=1)

In [14]:
print(client)

<Client: 'tcp://127.0.0.1:33633' processes=4 threads=4, memory=15.61 GiB>


Dask recommends scattering the large inputs across the workers, though this makes little difference in execution time.

In [15]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    for j in range(num_cpus):
        big_future = client.scatter((image, filters[j % num_cpus]))
        future = client.submit(fconv, big_future)
duration_dask = time.time() - start
print("Dask duration = {:.1f}, speedup = {:.2f}"
      .format(duration_dask, duration_conv*num_cpus / duration_dask))

2024-08-29 17:12:47,714 - distributed.worker - ERROR - Compute Failed
Key:       fconv-0b665a8d4891c7cc49bd947f93144994
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.07334627, -0.62169945, -0.67735824,  1.48857895],
       [ 0.17139794, -0.01512949,  0.48277583, -1.71977401],
       [ 0.16319201,  1.32872556,  0.26227987,  1.88038312],
       [ 1.74864732, -0.34274886, -0.18010743,  0.378581  ]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 17:12:47,812 - distributed.worker - ERROR - Compute Failed
Key:       fconv-06f486365baad91d307907b494b5910c
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-29 17:12:47,987 - distributed.worker - ERROR - Compute Failed
Key:       fconv-4e16b98a301aef8a19bef5b54acfead6
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.40267806,  2.09176739, -0.1468789 , -0.89278551],
       [-0.82003068,  0.64440009,  2.10031364,  0.32212676],
       [-1.30924933,  1.29362132, -0.16030545,  0.26563461],
       [ 0.28333749,  1.48944833,  0.40621051, -0.13003722]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 17:12:48,076 - distributed.worker - ERROR - Compute Failed
Key:       fconv-191a282a5d7c21e3eb3bf7c5b9368d58
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-29 17:12:48,255 - distributed.worker - ERROR - Compute Failed
Key:       fconv-0ad1331ab88df0f9b6c5995ebc539dca
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 1.49124383, -1.69967083, -0.02985792, -0.92183082],
       [-1.90595111, -0.37989453,  1.71336752, -0.95149078],
       [-0.10395455,  0.45867454, -0.43603419, -0.48393033],
       [-0.87247587, -1.36004964,  0.01432426,  0.01904892]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 17:12:48,338 - distributed.worker - ERROR - Compute Failed
Key:       fconv-7f41bc7ac3790b7079a9a086f7f1070d
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-29 17:12:48,515 - distributed.worker - ERROR - Compute Failed
Key:       fconv-c68ba9f317f98a1fb04826730dcec15c
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 1.03087196, -1.40536203, -1.33741389, -0.88369308],
       [-0.63218804,  0.8637891 , -1.16415582, -0.21606567],
       [-0.5304944 ,  0.13967581, -0.52941629, -0.04152657],
       [ 0.27602311,  0.35438379,  0.42993388,  0.94277641]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 17:12:48,602 - distributed.worker - ERROR - Compute Failed
Key:       fconv-b74085d85fef182ae3f06ce558e78cf0
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-29 17:12:48,774 - distributed.worker - ERROR - Compute Failed
Key:       fconv-45cc7e2800b45b6b166b9c3157c6237e
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.07334627, -0.62169945, -0.67735824,  1.48857895],
       [ 0.17139794, -0.01512949,  0.48277583, -1.71977401],
       [ 0.16319201,  1.32872556,  0.26227987,  1.88038312],
       [ 1.74864732, -0.34274886, -0.18010743,  0.378581  ]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 17:12:48,859 - distributed.worker - ERROR - Compute Failed
Key:       fconv-0b2945a23127da7417c062a2ea27f379
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

...

2024-08-29 17:13:21,824 - distributed.worker - ERROR - Compute Failed
Key:       fconv-036fd35717edeab4812bc0efdb3bce4d
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 1.49124383, -1.69967083, -0.02985792, -0.92183082],
       [-1.90595111, -0.37989453,  1.71336752, -0.95149078],
       [-0.10395455,  0.45867454, -0.43603419, -0.48393033],
       [-0.87247587, -1.36004964,  0.01432426,  0.01904892]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 17:13:21,910 - distributed.worker - ERROR - Compute Failed
Key:       fconv-287c5a67825142ebeee70c0b218fe3ca
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-29 17:13:22,084 - distributed.worker - ERROR - Compute Failed
Key:       fconv-9eaa585490ac00771f68f31174110fa7
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 1.03087196, -1.40536203, -1.33741389, -0.88369308],
       [-0.63218804,  0.8637891 , -1.16415582, -0.21606567],
       [-0.5304944 ,  0.13967581, -0.52941629, -0.04152657],
       [ 0.27602311,  0.35438379,  0.42993388,  0.94277641]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 17:13:22,176 - distributed.worker - ERROR - Compute Failed
Key:       fconv-6d6c996b9aacecb50c5a6fd867e40a47
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-29 17:13:22,359 - distributed.worker - ERROR - Compute Failed
Key:       fconv-158c313f2cd87ef4b7a0eb475c38599e
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.07334627, -0.62169945, -0.67735824,  1.48857895],
       [ 0.17139794, -0.01512949,  0.48277583, -1.71977401],
       [ 0.16319201,  1.32872556,  0.26227987,  1.88038312],
       [ 1.74864732, -0.34274886, -0.18010743,  0.378581  ]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 17:13:22,450 - distributed.worker - ERROR - Compute Failed
Key:       fconv-aa1c6893ffc955e20da9fa987b317f8b
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

Dask duration = 35.6, speedup = 3.97


2024-08-29 17:13:22,616 - distributed.worker - ERROR - Compute Failed
Key:       fconv-b00c924977b2166c05d117d1dc7602c3
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.40267806,  2.09176739, -0.1468789 , -0.89278551],
       [-0.82003068,  0.64440009,  2.10031364,  0.32212676],
       [-1.30924933,  1.29362132, -0.16030545,  0.26563461],
       [ 0.28333749,  1.48944833,  0.40621051, -0.13003722]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 17:13:22,707 - distributed.worker - ERROR - Compute Failed
Key:       fconv-6d97ba92e6f6e037f20650e7314c002e
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

In [16]:
client.close()

## Conclusions

* Ray is the most effective at speeding up the convolution workload by fully utilizing all available processes
* Multiprocessing is second in effectiveness
* Dask delivers the least speedup; perhaps due to having only six processes on the dask.distributed client

## About this notebook

This notebook was developed by David Shupe (shupe@ipac.caltech.edu) in conjunction with Jessica Krick and the IRSA Science Platform team at IPAC.

## Citations

If you use these software packages in your work, please use the following citations:

* Dask: Dask Development Team (2016). Dask: Library for dynamic task scheduling. URL https://dask.org
* Ray: The Ray Development Team. URL https://docs.ray.io